[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/machinelearnear/open-hf-spaces-in-studiolab/blob/main/run_google_colab.ipynb)

# Download, clean, and prep all `InsideAirBnB` data
- https://www.machinelearnear.com/
- https://www.youtube.com/@machinelearnear
- http://insideairbnb.com/get-the-data/
- https://medium.com/@aarandaniel/airbnbs-in-buenos-aires-the-data-caa173d81aa7
- https://github.com/beluplanas/Airbnb_bsas/blob/main/draft_notebook.ipynb

## Download data

In [1]:
import requests
from bs4 import BeautifulSoup

def scrape_airbnb_buenos_aires_urls():
    # fetch HTML content
    url = 'http://insideairbnb.com/get-the-data'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the "Buenos Aires" section
    buenos_aires_section = soup.find('h3', string='Buenos Aires, Ciudad Autónoma de Buenos Aires, Argentina').find_next('table')

    # extract URLs
    urls = {}
    for row in buenos_aires_section.find_all('tr'):
        columns = row.find_all('td')
        if len(columns) > 1:
            file_name = columns[1].get_text(strip=True)
            file_url = columns[1].find('a')['href']
            urls[file_name] = file_url

    return urls

In [2]:
# run the function
urls = scrape_airbnb_buenos_aires_urls()
print(urls)

{'listings.csv.gz': 'http://data.insideairbnb.com/argentina/ciudad-autónoma-de-buenos-aires/buenos-aires/2023-12-26/data/listings.csv.gz', 'calendar.csv.gz': 'http://data.insideairbnb.com/argentina/ciudad-autónoma-de-buenos-aires/buenos-aires/2023-12-26/data/calendar.csv.gz', 'reviews.csv.gz': 'http://data.insideairbnb.com/argentina/ciudad-autónoma-de-buenos-aires/buenos-aires/2023-12-26/data/reviews.csv.gz', 'listings.csv': 'http://data.insideairbnb.com/argentina/ciudad-autónoma-de-buenos-aires/buenos-aires/2023-12-26/visualisations/listings.csv', 'reviews.csv': 'http://data.insideairbnb.com/argentina/ciudad-autónoma-de-buenos-aires/buenos-aires/2023-12-26/visualisations/reviews.csv', 'neighbourhoods.csv': 'http://data.insideairbnb.com/argentina/ciudad-autónoma-de-buenos-aires/buenos-aires/2023-12-26/visualisations/neighbourhoods.csv', 'neighbourhoods.geojson': 'http://data.insideairbnb.com/argentina/ciudad-autónoma-de-buenos-aires/buenos-aires/2023-12-26/visualisations/neighbourhoods

In [3]:
import os
import requests
import gzip
import shutil
from tqdm import tqdm

def download_and_uncompress_files(urls):
    # create the 'data' directory if it doesn't exist
    os.makedirs('raw_data', exist_ok=True)

    for file_name, url in tqdm(urls.items()):
        # extract file name from URL
        local_filename = url.split('/')[-1]
        local_path = os.path.join('raw_data', local_filename)

        # check if the file already exists
        if os.path.exists(local_path):
            print(f"File {local_filename} already exists. Skipping download.")
            continue

        # download and save the file
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): 
                    f.write(chunk)
        
        print(f"Downloaded {file_name} to {local_path}")

        # check if the file is a .gz file and uncompress it
        if local_filename.endswith('.gz'):
            uncompressed_filename = local_filename.replace('.csv.gz', '_full.csv')
            uncompressed_path = os.path.join('raw_data', uncompressed_filename)
            with gzip.open(local_path, 'rb') as f_in:
                with open(uncompressed_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            print(f"Uncompressed {local_filename} to {uncompressed_filename}")
            
            # delete the original .gz file
            os.remove(local_path)
            print(f"Deleted compressed file {local_filename}")

In [4]:
download_and_uncompress_files(urls)

  0%|          | 0/7 [00:00<?, ?it/s]

Downloaded listings.csv.gz to raw_data/listings.csv.gz


 14%|█▍        | 1/7 [00:00<00:05,  1.11it/s]

Uncompressed listings.csv.gz to listings_full.csv
Deleted compressed file listings.csv.gz
Downloaded calendar.csv.gz to raw_data/calendar.csv.gz


 29%|██▊       | 2/7 [00:06<00:17,  3.47s/it]

Uncompressed calendar.csv.gz to calendar_full.csv
Deleted compressed file calendar.csv.gz
Downloaded reviews.csv.gz to raw_data/reviews.csv.gz


100%|██████████| 7/7 [00:11<00:00,  1.71s/it]

Uncompressed reviews.csv.gz to reviews_full.csv
Deleted compressed file reviews.csv.gz
File listings.csv already exists. Skipping download.
File reviews.csv already exists. Skipping download.
File neighbourhoods.csv already exists. Skipping download.
File neighbourhoods.geojson already exists. Skipping download.


## Load and clean data

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Listings

In [6]:
listings = pd.read_csv('raw_data/listings_full.csv')

In [7]:
selected_columns = [
    'id', 'listing_url', 'last_scraped', 'neighbourhood_cleansed', 'latitude',
    'longitude', 'room_type', 'bathrooms_text', 'beds', 'price',
    'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_location',
    'review_scores_value'
]

In [8]:
filtered_listings = listings[selected_columns].reset_index(drop=True)
filtered_listings.head()

,id,listing_url,last_scraped,neighbourhood_cleansed,latitude,longitude,room_type,bathrooms_text,beds,price,number_of_reviews_l30d,review_scores_rating,review_scores_location,review_scores_value
0,11508,https://www.airbnb.com/rooms/11508,2023-12-29,Palermo,-34.581840,-58.424150,Entire home/apt,1 bath,1.0,"$96,921.00",1,4.79,4.92,4.87
1,14222,https://www.airbnb.com/rooms/14222,2023-12-28,Palermo,-34.586170,-58.410360,Entire home/apt,1 bath,1.0,"$32,123.00",1,4.77,4.87,4.75
2,15074,https://www.airbnb.com/rooms/15074,2023-12-29,Nuñez,-34.538920,-58.465990,Private room,1 bath,1.0,"$24,230.00",0,NaN,NaN,NaN
3,16695,https://www.airbnb.com/rooms/16695,2023-12-28,Monserrat,-34.614390,-58.376110,Entire home/apt,1 bath,1.0,"$40,303.00",0,4.28,4.39,4.41
4,20062,https://www.airbnb.com/rooms/20062,2023-12-28,Palermo,-34.581848,-58.441605,Entire home/apt,1.5 baths,2.0,"$138,876.00",4,4.93,4.93,4.78


#### Add `estimated_nights_booked_l30d` with the following assumptions:
- 50% review per booking ratio
- 3 nights minimum
- 70% max occupancy.

In [9]:
def categorize_occupancy(reviews_l30d):
    estimated_nights = (reviews_l30d / 0.50) * 3
    if estimated_nights > 14:
        return 'high'
    elif estimated_nights >= 7:
        return 'medium'
    else:
        return 'low'

filtered_listings['estimated_nights_booked_l30d'] = filtered_listings['number_of_reviews_l30d'].apply(categorize_occupancy)

#### Clean `bathrooms_text` and `beds`

In [10]:
# extract numeric part from the 'bathrooms_text' column
filtered_listings['bathrooms'] = filtered_listings['bathrooms_text'].str.extract(r'(\d+\.?\d*)').astype(float)
filtered_listings.drop(columns=['bathrooms_text'], inplace=True)

#### Add `estimated_price_per_night_in_USD` with the following assumptions:
- Price is expressed in ARS
- We use historical USD-ARS rates, but only take the official ones, not "MEP" or "Blue" values.
- Modified code from: https://github.com/bahramkhanlarov/Currency-Exchange-Rate-Scraper

In [11]:
import datetime
from datetime import timedelta, date

# convert the 'last_scraped' column to datetime
filtered_listings['last_scraped'] = pd.to_datetime(filtered_listings['last_scraped'])

# get the minimum and maximum dates
start_date = filtered_listings['last_scraped'].min().date()
end_date = filtered_listings['last_scraped'].max().date()

# initialize an empty DataFrame to store the results
ars_to_usd = pd.DataFrame()

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

for single_date in daterange(start_date, end_date):
    dfs = pd.read_html(f'https://www.xe.com/currencytables/?from=ARS&date={single_date.strftime("%Y-%m-%d")}')[0]
    dfs = dfs[dfs['Currency'] == 'USD']
    dfs['Date'] = single_date.strftime("%Y-%m-%d")
    ars_to_usd = pd.concat([ars_to_usd, dfs], ignore_index=True)

In [12]:
ars_to_usd.head()

,Currency,Name,Units per ARS,ARS per unit,Date
0,USD,US Dollar,0.001241,805.777048,2023-12-26
1,USD,US Dollar,0.001239,807.408077,2023-12-27
2,USD,US Dollar,0.001238,807.917240,2023-12-28
3,USD,US Dollar,0.001237,808.524217,2023-12-29


In [13]:
import math

# ensure 'last_scraped' in filtered_listings is a datetime.date
filtered_listings['last_scraped'] = pd.to_datetime(filtered_listings['last_scraped']).dt.date

# ensure 'Date' in ars_to_usd is a datetime.date
ars_to_usd['Date'] = pd.to_datetime(ars_to_usd['Date']).dt.date

# clean the "price" column in filtered_listings
filtered_listings['price'] = filtered_listings['price'].replace('[\$,]', '', regex=True).astype(float)

# initialize a list to store the estimated prices in USD
estimated_prices_in_usd = []

# iterate over each row in filtered_listings
for row in tqdm(filtered_listings.itertuples(), total=len(filtered_listings)):
    # find the matching 'ARS per unit' in ars_to_usd
    exchange_rate = ars_to_usd[ars_to_usd['Date'] == getattr(row, 'last_scraped')]['ARS per unit']
    
    if not exchange_rate.empty and not pd.isna(row.price) and not pd.isna(exchange_rate.iloc[0]):
        # calculate the estimated price in USD
        estimated_price_usd = math.ceil(row.price / exchange_rate.iloc[0])
    else:
        # if no matching date is found, set a default value or NaN
        estimated_price_usd = float('nan')
    
    # append the calculated price to the list
    estimated_prices_in_usd.append(estimated_price_usd)

# assign the list as a new column in filtered_listings
filtered_listings['estimated_price_per_night_in_USD'] = estimated_prices_in_usd

100%|██████████| 33490/33490 [00:18<00:00, 1837.02it/s]


In [14]:
filtered_listings.head()

,id,listing_url,last_scraped,neighbourhood_cleansed,latitude,longitude,room_type,beds,price,number_of_reviews_l30d,review_scores_rating,review_scores_location,review_scores_value,estimated_nights_booked_l30d,bathrooms,estimated_price_per_night_in_USD
0,11508,https://www.airbnb.com/rooms/11508,2023-12-29,Palermo,-34.581840,-58.424150,Entire home/apt,1.0,96921.0,1,4.79,4.92,4.87,low,1.0,120.0
1,14222,https://www.airbnb.com/rooms/14222,2023-12-28,Palermo,-34.586170,-58.410360,Entire home/apt,1.0,32123.0,1,4.77,4.87,4.75,low,1.0,40.0
2,15074,https://www.airbnb.com/rooms/15074,2023-12-29,Nuñez,-34.538920,-58.465990,Private room,1.0,24230.0,0,NaN,NaN,NaN,low,1.0,30.0
3,16695,https://www.airbnb.com/rooms/16695,2023-12-28,Monserrat,-34.614390,-58.376110,Entire home/apt,1.0,40303.0,0,4.28,4.39,4.41,low,1.0,50.0
4,20062,https://www.airbnb.com/rooms/20062,2023-12-28,Palermo,-34.581848,-58.441605,Entire home/apt,2.0,138876.0,4,4.93,4.93,4.78,high,1.5,172.0


### Reviews

In [21]:
reviews = pd.read_csv('raw_data/reviews_full.csv')
reviews.drop(columns=['reviewer_id','reviewer_name'], inplace=True)

## Save to a local folder

In [24]:
os.makedirs('processed', exist_ok=True)

# save the DataFrames to CSV files in the 'processed' directory
filtered_listings.to_csv('processed/airbnb_listings.csv', index=False)
reviews.to_csv('processed/airbnb_reviews.csv', index=False)

```python
https://www.booking.com/searchresults.en-gb.html?ss=Colegiales%2C+Buenos+Aires%2C+Argentina%2C+South+America&ssne=&ssne_untouched=&label=gen173rf-1FCAQoggI49ANIM1gDaFCIAQGYAQm4AQfIAQzYAQHoAQH4AQOIAgGiAgxob29kbWFwcy5jb22oAgO4AvKUla0GwAIB0gIkZTc0YzZlYWEtOGQ5Yy00MTY1LWJlZDQtMmE0NDI5MWYwYTU02AIF4AIB&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=15755&dest_type=district&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b72a6a79f6b5033f&ac_meta=GhBiNzJhNmE3OWY2YjUwMzNmIAAoATICZW46CmNvbGVnaWFsZXNAAEoAUAA%3D&checkin=2024-02-01&checkout=2024-02-29&group_adults=2&no_rooms=1&group_children=0
```

In [16]:
# this works:
# https://www.booking.com/searchresults.en-gb.html?latitude=-34.5745;longitude=-58.451499999999996;&dest_type=district&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b72a6a79f6b5033f&ac_meta=GhBiNzJhNmE3OWY2YjUwMzNmIAAoATICZW46CmNvbGVnaWFsZXNAAEoAUAA%3D&checkin=2024-02-01&checkout=2024-02-29&group_adults=2&no_rooms=1&group_children=0

In [17]:
# a = "https://www.booking.com/searchresults.en-gb.html?ss=Colegiales%2C+Buenos+Aires%2C+Argentina%2C+South+America&ssne=&ssne_untouched=&label=gen173rf-1FCAQoggI49ANIM1gDaFCIAQGYAQm4AQfIAQzYAQHoAQH4AQOIAgGiAgxob29kbWFwcy5jb22oAgO4AvKUla0GwAIB0gIkZTc0YzZlYWEtOGQ5Yy00MTY1LWJlZDQtMmE0NDI5MWYwYTU02AIF4AIB&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=15755&dest_type=district&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b72a6a79f6b5033f&ac_meta=GhBiNzJhNmE3OWY2YjUwMzNmIAAoATICZW46CmNvbGVnaWFsZXNAAEoAUAA%3D&checkin=2024-02-01&checkout=2024-02-29&group_adults=2&no_rooms=1&group_children=0"